In [14]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

In [15]:
from llama_index.core import SimpleDirectoryReader

input_dir_path = "/Users/aagamchhajer/Desktop/aagam-projects/DeepSeek-RAG/pdf_dir"
loader = SimpleDirectoryReader(
    input_dir = input_dir_path,
    required_exts = [".pdf"],
    recursive=True
)
docs = loader.load_data()

In [ ]:

# setup llm & embedding model
llm=Ollama(model="deepseek-r1:8b", request_timeout=120.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)